In [1]:
use mavenfuzzyfactory

Commands completed successfully

Total execution time: 00:00:00.004

**Subiect: Sales Trends**

<span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">We're about to launch a new product, and I'd like to do a&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">deep dive on our current flagship product.&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Can you please pull monthly trends to date for number of&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">sales, total revenue, and total margin generated for the&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">business?</span>

In [2]:
show tables

(6 row(s) affected)

Total execution time: 00:00:01.022

Tables_in_mavenfuzzyfactory
order_item_refunds
order_items
orders
products
website_pageviews
website_sessions


In [3]:
select * from orders limit 3

(3 row(s) affected)

Total execution time: 00:00:01.007

order_id,created_at,website_session_id,user_id,primary_product_id,items_purchased,price_usd,cogs_usd
1,2012-03-19 10:42:46,20,20,1,1,49.99,19.49
2,2012-03-19 19:27:37,104,104,1,1,49.99,19.49
3,2012-03-20 06:44:45,147,147,1,1,49.99,19.49


In [4]:
select YEAR(date(created_at)) as yr, MONTH(DATE(created_at)) as mo,
count(order_id) as sales, sum(price_usd) as revenue, 
sum(price_usd - cogs_usd) as margin
from orders
where created_at < '2013-01-04'
GROUP by 1,2

(11 row(s) affected)

Total execution time: 00:00:01.029

yr,mo,sales,revenue,margin
2012,3,60,2999.40,1830.00
2012,4,99,4949.01,3019.50
2012,5,108,5398.92,3294.00
2012,6,140,6998.60,4270.00
2012,7,169,8448.31,5154.50
2012,8,228,11397.72,6954.00
2012,9,287,14347.13,8753.50
2012,10,371,18546.29,11315.50
2012,11,618,30893.82,18849.00
2012,12,506,25294.94,15433.00


**Subject: Impact of New Product Launch**

We launched our second product back on January 6th. Can <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">you pull together some trended analysis?&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">I'd like to see monthly order volume, overall conversion&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">rates, revenue per session, and a breakdown of sales by&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">product, all for the time period since April 1, 2012.</span>

In [5]:
select 
YEAR(date(website_sessions.created_at)) as yr,
MONTH(date(website_sessions.created_at)) as mo,
count(DISTINCT website_sessions.website_session_id) as sessions,
count(DISTINCT orders.order_id) as orders,
count(DISTINCT orders.order_id)/count(website_sessions.website_session_id) as conv_rate,
sum(orders.price_usd)/count(website_sessions.website_session_id) as rev_per_session,
count(DISTINCT case when orders.primary_product_id = 1 then orders.order_id else null end) as prod_1,
count(DISTINCT case when orders.primary_product_id = 2 then orders.order_id else null end) as prod_2
FROM website_sessions
LEFT JOIN orders 
ON website_sessions.website_session_id = orders.website_session_id
where website_sessions.created_at BETWEEN '2012-04-01' AND '2013-04-05'
GROUP BY 1,2

(13 row(s) affected)

Total execution time: 00:00:01.605

yr,mo,sessions,orders,conv_rate,rev_per_session,prod_1,prod_2
2012,4,3734,99,0.0265,1.325391,99,0
2012,5,3736,108,0.0289,1.445107,108,0
2012,6,3963,140,0.0353,1.765985,140,0
2012,7,4249,169,0.0398,1.988305,169,0
2012,8,6097,228,0.0374,1.869398,228,0
2012,9,6546,287,0.0438,2.191740,287,0
2012,10,8183,371,0.0453,2.266441,371,0
2012,11,14011,618,0.0441,2.204969,618,0
2012,12,10072,506,0.0502,2.511412,506,0
2013,1,6401,391,0.0611,3.127025,344,47


**Subject: Help w/ User Pathing**

Now that we have a new product, I'm thinking about our <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">user path and conversion funnel. Let's look at sessions which&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">hit the /products page and see where they went next.&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">Could you please pull clickthrough rates from /products&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">since the new product launch on January 6th 2013, by&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">product, and compare to the 3 months leading up to launch&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">as a baseline?</span>

In [9]:
-- Step 1: finding the /products pageviews we care about
CREATE TEMPORARY TABLE products_pageviews
SELECT
website_session_id,
website_pageview_id,
created_at,
CASE
WHEN created_at < '2013-01-06' THEN 'A. Pre_Product_2'
WHEN created_at >= '2013-01-06' THEN 'B. Post_Product_2'
ELSE 'uh oh...check logic'
END AS time_period
FROM website_pageviews
WHERE created_at < '2013-04-06'
AND created_at > '2012-10-06'
AND pageview_url = '/products' ;

Commands completed successfully

Total execution time: 00:00:00.681

In [14]:
-- Step 2: find the next pageview id that occurs AFTER the product pageview
CREATE TEMPORARY TABLE sessions_w_next_pageview_id
SELECT
products_pageviews. time_period,
products_pageviews. website_session_id,
MIN(website_pageviews.website_pageview_id) AS min_next_pageview_id
FROM products_pageviews
LEFT JOIN website_pageviews
ON website_pageviews.website_session_id = products_pageviews.website_session_id
AND website_pageviews.website_pageview_id > products_pageviews.website_pageview_id
GROUP BY 1,2;

Commands completed successfully

Total execution time: 00:00:00.216

In [15]:
select * from sessions_w_next_pageview_id limit 2

(2 row(s) affected)

Total execution time: 00:00:01.007

time_period,website_session_id,min_next_pageview_id
A. Pre_Product_2,31517,67217
A. Pre_Product_2,31518,67221


In [29]:
create temporary table sessions_w_next_pageview_url
select sessions_w_next_pageview_id.time_period, 
sessions_w_next_pageview_id.website_session_id,
website_pageviews.pageview_url as next_pageview_url
from sessions_w_next_pageview_id
join website_pageviews
ON sessions_w_next_pageview_id.min_next_pageview_id = website_pageviews.website_pageview_id

Commands completed successfully

Total execution time: 00:00:00.131

In [30]:
SELECT time_period,
COUNT(DISTINCT website_session_id) AS sessions,
COUNT(DISTINCT CASE WHEN next_pageview_url IS NOT NULL THEN website_session_id ELSE NULL END) AS w_next_pg,
COUNT(DISTINCT CASE WHEN next_pageview_url IS NOT NULL THEN website_session_id ELSE NULL END)/COUNT(DISTINCT website_session_id) AS pct_w_next_pg,
COUNT(DISTINCT CASE WHEN next_pageview_url = '/the-original-mr-fuzzy' THEN website_session_id ELSE NULL END) AS to_mrfuzzy,
COUNT(DISTINCT CASE WHEN next_pageview_url = '/the-original-mr-fuzzy' THEN website_session_id ELSE NULL END) /COUNT(DISTINCT website_session_id) AS pct_to_mrfuzzy,
COUNT(DISTINCT CASE WHEN next_pageview_url = '/the-forever-love-bear' THEN website_session_id ELSE NULL END) AS to_lovebear,
COUNT(DISTINCT CASE WHEN next_pageview_url = '/the-forever-love-bear' THEN website_session_id ELSE NULL END)/COUNT(DISTINCT website_session_id) AS pct_to_lovebear
FROM sessions_w_next_pageview_url
GROUP BY time_period

(2 row(s) affected)

Total execution time: 00:00:01.089

time_period,sessions,w_next_pg,pct_w_next_pg,to_mrfuzzy,pct_to_mrfuzzy,to_lovebear,pct_to_lovebear
A. Pre_Product_2,11347,11347,1.0000,11347,1.0000,0,0.0000
B. Post_Product_2,8200,8200,1.0000,6654,0.8115,1546,0.1885


**Subject: Product Conversion Funnels**

I'd like to look at our two products since January 6th and <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">analyze the conversion funnels from each product page to&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">conversion.&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">It would be great if you could produce a comparison between&nbsp;</span> <span style="font-family: -apple-system, BlinkMacSystemFont, sans-serif; color: var(--vscode-foreground);">the two conversion funnels, for all website traffic.</span>

In [37]:
select DISTINCT pageview_url from website_pageviews

(16 row(s) affected)

Total execution time: 00:00:01.665

pageview_url
/home
/products
/the-original-mr-fuzzy
/cart
/shipping
/billing
/thank-you-for-your-order
/lander-1
/billing-2
/the-forever-love-bear


In [65]:
-- drop TEMPORARY table sessions_to_product;
create TEMPORARY table sessions_to_product
select website_session_id, website_pageview_id, pageview_url
from website_pageviews
where pageview_url in ('/the-original-mr-fuzzy', '/the-forever-love-bear')
and website_pageviews.created_at > '2013-01-06'
and website_pageviews.created_at < '2013-04-10'

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.763

In [69]:
select * from sessions_to_product limit 10

(10 row(s) affected)

Total execution time: 00:00:01.004

website_session_id,website_pageview_id,pageview_url
63513,138944,/the-original-mr-fuzzy
63515,138952,/the-original-mr-fuzzy
63516,138956,/the-original-mr-fuzzy
63517,138959,/the-original-mr-fuzzy
63518,138962,/the-original-mr-fuzzy
63519,138966,/the-original-mr-fuzzy
63520,138969,/the-original-mr-fuzzy
63521,138972,/the-original-mr-fuzzy
63526,138986,/the-original-mr-fuzzy
63527,138989,/the-original-mr-fuzzy


In [85]:
-- drop temporary table user_journey;
create temporary table user_journey
select sessions_to_product.pageview_url,
sessions_to_product.website_session_id,
case when  website_pageviews.pageview_url = '/cart' then 1 else 0 end as to_cart,
case when  website_pageviews.pageview_url = '/shipping' then 1 else 0 end as to_shipping,
case when  website_pageviews.pageview_url = '/billing-2' then 1 else 0 end as to_billing,
case when  website_pageviews.pageview_url = '/thank-you-for-your-order' then 1 else 0 end as to_thankyou
from website_pageviews 
join sessions_to_product 
on website_pageviews.website_session_id = sessions_to_product.website_session_id
and website_pageviews.website_pageview_id > sessions_to_product.website_pageview_id -- other pages rather than products

Commands completed successfully

Commands completed successfully

Total execution time: 00:00:00.271

In [87]:
select * from user_journey limit 10

(10 row(s) affected)

Total execution time: 00:00:01.006

pageview_url,website_session_id,to_cart,to_shipping,to_billing,to_thankyou
/the-original-mr-fuzzy,63513,1,0,0,0
/the-original-mr-fuzzy,63513,0,1,0,0
/the-original-mr-fuzzy,63513,0,0,1,0
/the-original-mr-fuzzy,63513,0,0,0,1
/the-original-mr-fuzzy,63515,1,0,0,0
/the-original-mr-fuzzy,63518,1,0,0,0
/the-original-mr-fuzzy,63521,1,0,0,0
/the-original-mr-fuzzy,63521,0,1,0,0
/the-original-mr-fuzzy,63521,0,0,1,0
/the-original-mr-fuzzy,63521,0,0,0,1


In [90]:
select pageview_url,
count(DISTINCT website_session_id) as session,
count(DISTINCT case when to_cart = 1 then website_session_id else null end) as to_cart,
count(DISTINCT case when to_shipping = 1 then website_session_id else null end) as to_shipping,
count(DISTINCT case when to_billing = 1 then website_session_id else null end) as to_billing,
count(DISTINCT case when to_thankyou = 1 then website_session_id else null end) as to_thankyou
from user_journey
GROUP by 1

(2 row(s) affected)

Total execution time: 00:00:01.037

pageview_url,session,to_cart,to_shipping,to_billing,to_thankyou
/the-forever-love-bear,877,877,603,488,301
/the-original-mr-fuzzy,3038,3038,2084,1710,1088
